In [20]:
import numpy as np
import pydicom as dicom
import matplotlib.pyplot as plt
from ipywidgets import interact
import json
import scipy.interpolate
import pandas as pd

In [1]:
import sys
sys.path.insert(0, "../src")


In [23]:
import bdfileutils as bdf
import bd4cnn.bd4cnn_metrics

In [24]:
import importlib
importlib.reload(bd4cnn.bd4cnn_metrics)

<module 'bd4cnn.bd4cnn_metrics' from '../src\\bd4cnn\\bd4cnn_metrics.py'>

In [50]:
pids = [
    '1405662',
    '1405045',
    '1404100',
    '1403302',
    '1402847',
    '1402419',
    '1402186',
    '1401571',
    '1400518',
    '1399784',
    '1399105',
    '1398889',
    '1398806',
    '1398511',
    '1397534'
]
df = pd.DataFrame(columns=['PID', 'TARGET_DOSE', 'ROI', 'TYPE', 'PTV MIN', 'PTV AVG', 'PTV MAX'])
for idx, pid in enumerate(pids):
    args = lambda x:x
    folder = f'E:\\sim4\\{pid}'
    args.root_folder = folder
    args.roi_subfolder = 'output_50'
    #args.doses_subfolder_file = "output_50_e_31+41\doses_array.nparray"
    args.doses_subfolder_file = "output_50\doses_to_max.nparray"
    args.single = True
    args.doses_dtype='f4'
    #args.doses_dtype='u1'

    res = bd4cnn.bd4cnn_metrics.do_run(args)

   # print(m[folder])
    for r, m in res[folder]['metrics'].items():
        if 'blizna PTV' in r:
            df.loc[idx] = [pid, res[folder]['target_dose'],  r, 'blizna', m['min'], m['avg'], m['max']]
        elif 'piers PTV' in r:
            df.loc[idx] = [pid, res[folder]['target_dose'], r, 'piers', m['min'], m['avg'], m['max']]
            
df

2021-02-04 12:37:53,001 [INFO]: Reading ndarray data from file: E:\sim4\1405662\output_50\rois_marks_original.nparray
2021-02-04 12:37:53,030 [INFO]: Reading ndarray data from file: E:\sim4\1405662\output_50\doses_to_max.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [    262144]  0.00 |   8.30 |   0.48 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [    131072]  0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  48.77 |   7.25 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [         2] 40.47 |  48.77 |  44.99 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 12:37:54,678 [INFO]: Reading ndarray data from file: E:\sim4\1405045\output_50\rois_marks_original.nparray
2021-02-04 12:37:55,280 [INFO]: Reading ndarray data from file: E:\sim4\1405045\output_50\doses_to_max.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 54.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                  CouchSurface [2147483648] 41.53 |  47.46 |  45.25 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  55.77 |   7.90 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                         _4500 [ 268435456]  0.00 |  20.17 |   0.36 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                         boost [         2]  0.00 |  48.65 |  44.81 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 12:37:57,156 [INFO]: Reading ndarray data from file: E:\sim4\1404100\output_50\rois_marks_original.nparray
2021-02-04 12:37:57,796 [INFO]: Reading ndarray data from file: E:\sim4\1404100\output_50\doses_to_max.nparray


...

2021-02-04 12:38:23,176 [INFO]: Reading ndarray data from file: E:\sim4\1398511\output_50\rois_marks_original.nparray
2021-02-04 12:38:23,978 [INFO]: Reading ndarray data from file: E:\sim4\1398511\output_50\doses_to_max.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [  33554432] 31.03 |  47.59 |  44.73 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [  16777216]  0.00 |   5.17 |   0.42 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  47.59 |   6.28 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [         2]  1.03 |  46.55 |  17.73 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 12:38:26,237 [INFO]: Reading ndarray data from file: E:\sim4\1397534\output_50\rois_marks_original.nparray
2021-02-04 12:38:26,866 [INFO]: Reading ndarray data from file: E:\sim4\1397534\output_50\doses_to_max.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [   4194304]  0.00 |  46.46 |  43.35 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [   2097152]  0.00 |  47.52 |  41.96 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  47.52 |   7.12 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [      1024]  1.06 |  47.52 |   7.32 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

,PID,TARGET_DOSE,ROI,TYPE,PTV MIN,PTV AVG,PTV MAX
0,1405662,45.0,blizna PTV,blizna,33.206253,44.765133,48.771683
1,1405045,54.0,piers PTV-plan,piers,0.000000,4.139217,42.715187
2,1404100,50.0,blizna PTV,blizna,0.000000,39.427170,53.270882
3,1403302,56.0,piers PTV,piers,35.748550,45.182114,49.308350
4,1402847,45.0,blizna PTV,blizna,42.675682,45.131004,47.880035
5,1402419,45.0,blizna PTV,blizna,34.735294,44.733456,49.471478
6,1402186,56.0,piers PTV-plan,piers,32.231171,46.281521,58.264038
7,1401571,45.0,piers PTV-plan,piers,1.048578,14.076074,48.234589
8,1400518,56.0,piers PTV-plan,piers,33.297153,45.540775,55.495258
9,1399784,55.0,piers PTV-plan,piers,34.719330,46.113297,57.466476


In [48]:
print("output_50_e_31+41")
df

output_50_e_31+41


,PID,TARGET_DOSE,ROI,TYPE,PTV MIN,PTV AVG,PTV MAX
0,1405662,45.0,blizna PTV,blizna,0.000000,44.048645,51.884770
1,1405045,54.0,piers PTV-plan,piers,0.000000,3.576930,55.767048
2,1404100,50.0,blizna PTV,blizna,0.000000,40.042413,59.189867
3,1403302,56.0,piers PTV,piers,2.465417,44.095409,56.704598
4,1402847,45.0,blizna PTV,blizna,0.000000,43.737121,49.961774
5,1402419,45.0,blizna PTV,blizna,0.000000,44.650036,51.576650
6,1402186,56.0,piers PTV-plan,piers,0.000000,43.340984,63.222681
7,1401571,45.0,piers PTV-plan,piers,1.048578,14.926114,50.331748
8,1400518,56.0,piers PTV-plan,piers,0.000000,46.458492,59.194945
9,1399784,55.0,piers PTV-plan,piers,0.000000,46.005585,62.255352


In [51]:
print("output_50 - oryginalne z Elcipse")
df

output_50 - oryginalne z Elcipse


,PID,TARGET_DOSE,ROI,TYPE,PTV MIN,PTV AVG,PTV MAX
0,1405662,45.0,blizna PTV,blizna,33.206253,44.765133,48.771683
1,1405045,54.0,piers PTV-plan,piers,0.000000,4.139217,42.715187
2,1404100,50.0,blizna PTV,blizna,0.000000,39.427170,53.270882
3,1403302,56.0,piers PTV,piers,35.748550,45.182114,49.308350
4,1402847,45.0,blizna PTV,blizna,42.675682,45.131004,47.880035
5,1402419,45.0,blizna PTV,blizna,34.735294,44.733456,49.471478
6,1402186,56.0,piers PTV-plan,piers,32.231171,46.281521,58.264038
7,1401571,45.0,piers PTV-plan,piers,1.048578,14.076074,48.234589
8,1400518,56.0,piers PTV-plan,piers,33.297153,45.540775,55.495258
9,1399784,55.0,piers PTV-plan,piers,34.719330,46.113297,57.466476
